# San Francisco Police Deprt. Crime Data Analysis Pipeline

In a dynamic urban environment like San Francisco, analyzing crime data is essential for understanding crime patterns, identifying hotspots, and devising effective strategies to maintain public safety. This dataset provides a wealth of information related to various aspects of reported incidents, including incident date and time, incident category, location, resolution, and more. Exploring this dataset through dashboard will help us gain insights into crime trends, incident distribution, and police response.

The dataset's columns encompass a wide range of information, including incident timestamps, descriptions, categories, resolution status, geographical coordinates, police district, neighborhood, and more. This Analysis project focuses on conducting a comprehensive analysis of the crime dataset based in San Francisco. The dataset contains incident reports spanning from 2018 to 2023, by performing exploratory data analysis, we aim to uncover patterns, correlations, and anomalies within this dataset to support data-driven decision-making for law enforcement and city planning.

In [24]:
#importing necessary libraries
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [25]:
#reading the dataset
df = pd.read_csv("Dataset\Police_Department_Incident_Reports__2018_to_Present (2).csv")

In [26]:
#print of the sample of the dataframe
df.head()

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Row ID,Incident ID,Incident Number,CAD Number,Report Type Code,Report Type Description,Filed Online,Incident Code,Incident Category,Incident Subcategory,Incident Description,Resolution,Intersection,CNN,Police District,Analysis Neighborhood,Supervisor District,Supervisor District 2012,Latitude,Longitude,Point,Neighborhoods,ESNCAG - Boundary File,Central Market/Tenderloin Boundary Polygon - Updated,Civic Center Harm Reduction Project Boundary,HSOC Zones as of 2018-06-05,Invest In Neighborhoods (IIN) Areas,Current Supervisor Districts,Current Police Districts
0,2023/03/13 11:41:00 PM,2023/03/13,23:41,2023,Monday,2023/03/13 11:41:00 PM,125373607041,1253736,230167874,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023/03/01 05:02:00 AM,2023/03/01,05:02,2023,Wednesday,2023/03/11 03:40:00 PM,125379506374,1253795,236046151,NaN,II,Coplogic Initial,True,6374,Larceny Theft,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023/03/13 01:16:00 PM,2023/03/13,13:16,2023,Monday,2023/03/13 01:17:00 PM,125357107041,1253571,220343896,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023/03/13 10:59:00 AM,2023/03/13,10:59,2023,Monday,2023/03/13 11:00:00 AM,125355107041,1253551,230174885,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023/03/14 06:44:00 PM,2023/03/14,18:44,2023,Tuesday,2023/03/14 06:45:00 PM,125402407041,1254024,230176728,NaN,VS,Vehicle Supplement,NaN,7041,Recovered Vehicle,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#standardizing the Report code respective to the report type description
#Init dictionary
Report_Code_correction = {
    'Vehicle Supplement' : 'VS',
    'Coplogic Initial' : 'CI',
    'Coplogic Supplement' : 'CS',
    'Initial' : 'II',
    'Initial Supplement' : 'IS',
    'Vehicle Initial': 'VI'
}
df['Report_Type_Code'] = df['Report_Type_Description'].map(Report_Code_correction)

### Transformation in the raw dataset

In [29]:
#Changing names of columns to insert Underscore in spaces.
column_name = df.columns
column_name_changed = list()
for i in range(len(column_name)):
    column_name_changed.append(column_name[i].replace(" ","_"))
df.columns = column_name_changed


#standardizing the Report code respective to the report type description
#Init of the dictionary
Report_Code_correction = {
    'Vehicle Supplement' : 'VS',
    'Coplogic Initial' : 'CI',
    'Coplogic Supplement' : 'CS',
    'Initial' : 'II',
    'Initial Supplement' : 'IS',
    'Vehicle Initial': 'VI'
}
df['Report_Type_Code'] = df['Report_Type_Description'].map(Report_Code_correction)


#to standardize the values in the Incident Category column that represent the same concept
#Init of the dictionary
replace_dict = {
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft?': 'Vehicle Theft and Recovery',
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Larceny Theft': 'Property Crimes',
    'Lost Property': 'Property Crimes',
    'Stolen Property': 'Property Crimes',
    'Burglary': 'Property Crimes',
    'Vandalism': 'Property Crimes',
    'Drug Violation': 'Drug-related Incidents',
    'Drug Offense': 'Drug-related Incidents',
    'Non-Criminal': 'Miscellaneous',
    'Case Closure': 'Miscellaneous',
    'Other Miscellaneous': 'Miscellaneous',
    'Other Offenses': 'Miscellaneous',
    'Other': 'Miscellaneous',
    'Miscellaneous Investigation': 'Miscellaneous',
    'Suspicious Occ': 'Suspicious',
    'Suspicious': 'Suspicious',
    'Missing Person': 'Police Interaction Incidents',
    'Courtesy Report': 'Police Interaction Incidents',
    'Fraud': 'Fraudulent Activities',
    'Forgery And Counterfeiting': 'Fraudulent Activities',
    'Prostitution': 'Sex-related Incidents',
    'Human Trafficking, Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (A), Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (B), Involuntary Servitude': 'Sex-related Incidents',
    'Embezzlement': 'Financial and Regulatory Offenses',
    'Gambling': 'Financial and Regulatory Offenses',
    'Liquor Laws': 'Financial and Regulatory Offenses'
}
df['Incident_Category'].replace(replace_dict, inplace=True)


#To fill the missing values in the Incident Category by using the respective Incident Description
#Init of the dictionary
description_to_incident = {
    'Vehicle, Seizure Order Service': 'Vehicle Theft and Recovery',
    'Gun Violence Restraining Order': 'Weapons Offense',
    'Driving, Stunt Vehicle/Street Racing': 'Traffic Collision',
    'Cryptocurrency Related Crime (secondary code only)': 'Financial and Regulatory Offenses',
    'Auto Impounded': 'Vehicle Impounded',
    'Theft, Boat': 'Property Crimes',
    'Theft, Animal, Att.': 'Property Crimes',
    'SFMTA Muni Transit Operator-Bus/LRV': 'SFMTA',
    'Military Ordinance': 'Weapons Offense',
    'SFMTA Parking and Control Officer': 'SFMTA',
    'Cloned Cellular Phone, Use': 'Fraudulent Activities',
    'Public Health Order Violation, Notification': 'Miscellaneous',
    'Public Health Order Violation, After Notification': 'Miscellaneous',
    'Assault, Commission of While Armed': 'Assault',
    'Theft, Phone Booth, <$50': 'Property Crimes',
    'Gun Violence Restraining Order Violation': 'Weapons Offense',
    'Theft, Phone Booth, $200-$950': 'Property Crimes',
    'Theft, Phone Booth, $50-$200': 'Property Crimes',
    'Vehicle, Seizure Order': 'Vehicle Theft and Recovery',
    'SFMTA Employee-Non Operator/Station Agent-Other Employee': 'SFMTA',
    'Assault, By Police Officers': 'Assault',
    'Crimes Involving Receipts or Titles': 'Fraudulent Activities',
    'Procurement, Pimping, & Pandering': 'Sex-related Incidents',
    'Service of Documents Related to a Civil Drug Abatement and/or Public Nuisance Action': 'Miscellaneous',
    'Pyrotechnic Explosive Device - Barrel Bomb': 'Fire Report',
    'Theft, Phone Booth, >$950': 'Property Crimes'
}
df['Incident_Category'] = df['Incident_Category'].fillna(df['Incident_Description'].map(description_to_incident))


#converting columns to datetime format in df
df['Incident_Datetime']= pd.to_datetime(df['Incident_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')
df['Report_Datetime']= pd.to_datetime(df['Report_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')

#dropping the incomplete month data of September 2023
df = df.drop((df[(df['Incident_Datetime'].dt.month == 9) & (df['Incident_Year'] == 2023)]).index)

#Converting boolean column to binary column
df['Filed_Online'] = df['Filed_Online'].notnull().astype(int)

In [30]:
df.head()

,Incident_Datetime,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Report_Datetime,Row_ID,Incident_ID,Incident_Number,CAD_Number,Report_Type_Code,Report_Type_Description,Filed_Online,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Resolution,Intersection,CNN,Police_District,Analysis_Neighborhood,Supervisor_District,Supervisor_District_2012,Latitude,Longitude,Point,Neighborhoods,ESNCAG_-_Boundary_File,Central_Market/Tenderloin_Boundary_Polygon_-_Updated,Civic_Center_Harm_Reduction_Project_Boundary,HSOC_Zones_as_of_2018-06-05,Invest_In_Neighborhoods_(IIN)_Areas,Current_Supervisor_Districts,Current_Police_Districts
0,2023-03-13 23:41:00,2023/03/13,23:41,2023,Monday,2023-03-13 23:41:00,125373607041,1253736,230167874,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-03-01 05:02:00,2023/03/01,05:02,2023,Wednesday,2023-03-11 15:40:00,125379506374,1253795,236046151,NaN,CI,Coplogic Initial,1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-03-13 13:16:00,2023/03/13,13:16,2023,Monday,2023-03-13 13:17:00,125357107041,1253571,220343896,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-03-13 10:59:00,2023/03/13,10:59,2023,Monday,2023-03-13 11:00:00,125355107041,1253551,230174885,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-03-14 18:44:00,2023/03/14,18:44,2023,Tuesday,2023-03-14 18:45:00,125402407041,1254024,230176728,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df = df.drop((df[(df['Incident_Datetime'].dt.month == 9) & (df['Incident_Year'] == 2023)]).index)

df['Id'] = df.index

In [399]:
column_name_changed

['Incident_Datetime',
 'Incident_Date',
 'Incident_Time',
 'Incident_Year',
 'Incident_Day_of_Week',
 'Report_Datetime',
 'Row_ID',
 'Incident_ID',
 'Incident_Number',
 'CAD_Number',
 'Report_Type_Code',
 'Report_Type_Description',
 'Filed_Online',
 'Incident_Code',
 'Incident_Category',
 'Incident_Subcategory',
 'Incident_Description',
 'Resolution',
 'Intersection',
 'CNN',
 'Police_District',
 'Analysis_Neighborhood',
 'Supervisor_District',
 'Supervisor_District_2012',
 'Latitude',
 'Longitude',
 'Point',
 'Neighborhoods',
 'ESNCAG_-_Boundary_File',
 'Central_Market/Tenderloin_Boundary_Polygon_-_Updated',
 'Civic_Center_Harm_Reduction_Project_Boundary',
 'HSOC_Zones_as_of_2018-06-05',
 'Invest_In_Neighborhoods_(IIN)_Areas',
 'Current_Supervisor_Districts',
 'Current_Police_Districts']

In [400]:
df.isnull().sum()

Incident_Datetime                                            0
Incident_Date                                                0
Incident_Time                                                0
Incident_Year                                                0
Incident_Day_of_Week                                         0
Report_Datetime                                              0
Row_ID                                                       0
Incident_ID                                                  0
Incident_Number                                              0
CAD_Number                                              174672
Report_Type_Code                                             0
Report_Type_Description                                      0
Filed_Online                                                 0
Incident_Code                                                0
Incident_Category                                            0
Incident_Subcategory                                   

In [401]:
df.dtypes

Incident_Datetime                                       datetime64[ns]
Incident_Date                                                   object
Incident_Time                                                   object
Incident_Year                                                    int64
Incident_Day_of_Week                                            object
Report_Datetime                                         datetime64[ns]
Row_ID                                                           int64
Incident_ID                                                      int64
Incident_Number                                                  int64
CAD_Number                                                     float64
Report_Type_Code                                                object
Report_Type_Description                                         object
Filed_Online                                                     int32
Incident_Code                                                    int64
Incide

replace_dict = {
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft': 'Vehicle Theft and Recovery',
    'Motor Vehicle Theft?': 'Vehicle Theft and Recovery',
    'Recovered Vehicle': 'Vehicle Theft and Recovery',
    'Larceny Theft': 'Property Crimes',
    'Lost Property': 'Property Crimes',
    'Stolen Property': 'Property Crimes',
    'Burglary': 'Property Crimes',
    'Vandalism': 'Property Crimes',
    'Drug Violation': 'Drug-related Incidents',
    'Drug Offense': 'Drug-related Incidents',
    'Non-Criminal': 'Miscellaneous',
    'Case Closure': 'Miscellaneous',
    'Other Miscellaneous': 'Miscellaneous',
    'Other Offenses': 'Miscellaneous',
    'Other': 'Miscellaneous',
    'Miscellaneous Investigation': 'Miscellaneous',
    'Suspicious Occ': 'Suspicious',
    'Suspicious': 'Suspicious',
    'Missing Person': 'Police Interaction Incidents',
    'Courtesy Report': 'Police Interaction Incidents',
    'Fraud': 'Fraudulent Activities',
    'Forgery And Counterfeiting': 'Fraudulent Activities',
    'Prostitution': 'Sex-related Incidents',
    'Human Trafficking, Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (A), Commercial Sex Acts': 'Sex-related Incidents',
    'Human Trafficking (B), Involuntary Servitude': 'Sex-related Incidents',
    'Embezzlement': 'Financial and Regulatory Offenses',
    'Gambling': 'Financial and Regulatory Offenses',
    'Liquor Laws': 'Financial and Regulatory Offenses'
}

df['Incident_Category'].replace(replace_dict, inplace=True)

description_to_incident = {
    'Vehicle, Seizure Order Service': 'Vehicle Theft and Recovery',
    'Gun Violence Restraining Order': 'Weapons Offense',
    'Driving, Stunt Vehicle/Street Racing': 'Traffic Collision',
    'Cryptocurrency Related Crime (secondary code only)': 'Financial and Regulatory Offenses',
    'Auto Impounded': 'Vehicle Impounded',
    'Theft, Boat': 'Property Crimes',
    'Theft, Animal, Att.': 'Property Crimes',
    'SFMTA Muni Transit Operator-Bus/LRV': 'SFMTA',
    'Military Ordinance': 'Weapons Offense',
    'SFMTA Parking and Control Officer': 'SFMTA',
    'Cloned Cellular Phone, Use': 'Fraudulent Activities',
    'Public Health Order Violation, Notification': 'Miscellaneous',
    'Public Health Order Violation, After Notification': 'Miscellaneous',
    'Assault, Commission of While Armed': 'Assault',
    'Theft, Phone Booth, <$50': 'Property Crimes',
    'Gun Violence Restraining Order Violation': 'Weapons Offense',
    'Theft, Phone Booth, $200-$950': 'Property Crimes',
    'Theft, Phone Booth, $50-$200': 'Property Crimes',
    'Vehicle, Seizure Order': 'Vehicle Theft and Recovery',
    'SFMTA Employee-Non Operator/Station Agent-Other Employee': 'SFMTA',
    'Assault, By Police Officers': 'Assault',
    'Crimes Involving Receipts or Titles': 'Fraudulent Activities',
    'Procurement, Pimping, & Pandering': 'Sex-related Incidents',
    'Service of Documents Related to a Civil Drug Abatement and/or Public Nuisance Action': 'Miscellaneous',
    'Pyrotechnic Explosive Device - Barrel Bomb': 'Fire Report',
    'Theft, Phone Booth, >$950': 'Property Crimes'
}

#df['Incident Category'] = df['Incident Description'].map(incident_to_category)
df['Incident_Category'] = df['Incident_Category'].fillna(df['Incident_Description'].map(description_to_incident))

#converting columns to datetime format in df
df['Incident_Datetime']= pd.to_datetime(df['Incident_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')
df['Report_Datetime']= pd.to_datetime(df['Report_Datetime'], format= '%Y/%m/%d %I:%M:%S %p')

In [402]:
print("Incident oldest date",min(df['Incident_Datetime']))
print("Incident latest date",max(df['Incident_Datetime']))

Incident oldest date 2018-01-01 00:00:00
Incident latest date 2023-09-14 22:22:00


In [403]:
print("Report oldest date",min(df['Report_Datetime']))
print("Report latest date",max(df['Report_Datetime']))

Report oldest date 2018-01-01 00:01:00
Report latest date 2023-09-14 23:30:00


### Time Dimension table

In [404]:
#Init of a dataframe for the dimension table
df_time = pd.DataFrame(pd.date_range(start="2000-01-01", end="2000-01-02", freq="T", inclusive='left'))
df_time['Id'] = df_time.index
df_time['Time'] = df_time[0].dt.time
df_time.drop(columns=[0], inplace=True)
#df_time['time'] = pd.to_datetime(df_time['time'], format= '%I:%M:%S')
#df_time['time'] = df_time['time'].strftime('%I:%M:%S')

#To exrtact various tangents from the time columns.
df_time['Hour'] = df_time['Time'].astype(str).str[:2].astype(int)

In [405]:
df_time.head()

,Id,Time,Hour
0,0,00:00:00,0
1,1,00:01:00,0
2,2,00:02:00,0
3,3,00:03:00,0
4,4,00:04:00,0


### Date Dimension table

In [407]:
#Init of a dataframe for the dimension table
df_date = pd.DataFrame(pd.date_range(start=min(df['Incident_Datetime']), end=max(df['Incident_Datetime']), freq="D", inclusive='both'))
df_date['Id'] = df_date.index
df_date['Date'] = df_date[0].dt.date
df_date.drop(columns=[0], inplace=True)
df_date['Date'] = pd.to_datetime(df_date['Date'])

#To exrtact various tangents from the date columns.
df_date['Dayofmonth'] = df_date['Date'].dt.day
df_date['Month_Number'] = df_date['Date'].dt.month
df_date['Month_Name'] = df_date['Date'].dt.month_name()
df_date['Dayofweek_number'] = df_date['Date'].dt.day_of_week
df_date['Dayofweek_name'] = df_date['Date'].dt.day_name()
df_date['Year'] = df_date['Date'].dt.year

In [408]:
df_date.head()

,Id,Date,Dayofmonth,Month_Number,Month_Name,Dayofweek_number,Dayofweek_name,Year
0,0,2018-01-01,1,1,January,0,Monday,2018
1,1,2018-01-02,2,1,January,1,Tuesday,2018
2,2,2018-01-03,3,1,January,2,Wednesday,2018
3,3,2018-01-04,4,1,January,3,Thursday,2018
4,4,2018-01-05,5,1,January,4,Friday,2018


In [352]:
df_date.dtypes

Id                           int64
Date                datetime64[ns]
Dayofmonth                   int32
Month_Number                 int32
Month_Name                  object
Dayofweek_number             int32
Dayofweek_name              object
dtype: object

df_date['Day_of_monthj'] = df_date['date'].dt.day
df_date['Month_Number'] = df_date['date'].dt.month
df_date['Month_Name'] = df_date['date'].dt.month_name()
df_date['Day_of_the_week_number'] = df_date['date'].dt.dayofweek
df_date['Day_of_the_week'] = df_date['date'].dt.day_name()

df['Report Year'] = df['Report Datetime'].dt.year
df['Report Day of Week'] = df['Report Datetime'].dt.day_name()

df['Incident Month Number'] = df['Incident Datetime'].dt.month
df['Incident Month Name'] = df['Incident Datetime'].dt.month_name()
df['Day of the week number'] = df['Incident Datetime'].dt.dayofweek

### Report Info Dimension table

In [353]:
#df_report_info = df.groupby(['Report Type Code','Report Type Description'])['Id'].count()
#Init of a dataframe for the dimension table
df_report_info = df[['Report_Type_Code','Report_Type_Description']].copy()

df_report_info.drop_duplicates(inplace=True)
#df_report_info.drop(labels=[9], inplace=True)
df_report_info.reset_index(inplace=True)
df_report_info.drop(columns=['index'], inplace=True)

In [354]:
df_report_info

,Report_Type_Code,Report_Type_Description
0,VS,Vehicle Supplement
1,CI,Coplogic Initial
2,CS,Coplogic Supplement
3,II,Initial
4,IS,Initial Supplement
5,VI,Vehicle Initial


### Incident Info Dimension table

In [355]:
#Init of a dataframe for the dimension table
df_incident_info = df[['Incident_Code','Incident_Category','Incident_Subcategory','Incident_Description']].copy()

df_incident_info.drop_duplicates(inplace=True)
df_incident_info.drop(labels=[2029], inplace=True)
df_incident_info.reset_index(inplace=True)
df_incident_info.drop(columns=['index'], inplace=True)

In [356]:
df_incident_info

,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description
0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto"
1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950"
2,6372,Property Crimes,Larceny Theft - Other,"Theft, Other Property, $50-$200"
3,6244,Property Crimes,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950"
4,4134,Assault,Simple Assault,Battery
...,...,...,...,...
837,27071,Miscellaneous,Other,Impersonating Fire Department Member
838,5371,Property Crimes,Burglary - Other,"Burglary, Safe, Other, with Explosives"
839,16520,Drug-related Incidents,Drug Violation,"Barbiturates, Possession For Sale"
840,6354,Property Crimes,NaN,"Theft, Phone Booth, >$950"


df_incident_info = df[['Incident_Code','Incident_Category','Incident_Subcategory','Incident_Description']].copy()
df_incident_info.drop_duplicates(inplace=True)
#df_incident_info.iloc[2029]
df_incident_info[df_incident_info['Incident_Code'] == 7060]

#df_incident_info['Incident_Code'].value_counts()

In [357]:
df_incident_info['Incident_Code'].value_counts()

Incident_Code
7041     1
6399     1
19053    1
16060    1
12025    1
        ..
9015     1
3024     1
9026     1
10120    1
51010    1
Name: count, Length: 842, dtype: int64

In [358]:
df_incident_info

,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description
0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto"
1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950"
2,6372,Property Crimes,Larceny Theft - Other,"Theft, Other Property, $50-$200"
3,6244,Property Crimes,Larceny - From Vehicle,"Theft, From Locked Vehicle, >$950"
4,4134,Assault,Simple Assault,Battery
...,...,...,...,...
837,27071,Miscellaneous,Other,Impersonating Fire Department Member
838,5371,Property Crimes,Burglary - Other,"Burglary, Safe, Other, with Explosives"
839,16520,Drug-related Incidents,Drug Violation,"Barbiturates, Possession For Sale"
840,6354,Property Crimes,NaN,"Theft, Phone Booth, >$950"


### Police District Dimension table

In [359]:
#Init of a dataframe for the dimension table
df_police_district = df[['Police_District']].copy()
df_police_district.drop_duplicates(inplace=True)
df_police_district.reset_index(inplace=True)
df_police_district.drop(columns=['index'], inplace=True)
df_police_district['Police_district_Id'] = df_police_district.index
df_police_district = df_police_district[['Police_district_Id', 'Police_District']]


In [360]:
df_police_district

,Police_district_Id,Police_District
0,0,Out of SF
1,1,Mission
2,2,Central
3,3,Park
4,4,Bayview
5,5,Richmond
6,6,Southern
7,7,Northern
8,8,Taraval
9,9,Tenderloin


### Resolution Dimension table

In [361]:
#Init of a dataframe for the dimension table
df_resolution = df[['Resolution']].copy()
df_resolution.drop_duplicates(inplace=True)
df_resolution.reset_index(inplace=True)
df_resolution.drop(columns=['index'], inplace=True)
df_resolution['Resolution_Id'] = df_resolution.index
df_resolution = df_resolution[['Resolution_Id', 'Resolution']]

In [362]:
df_resolution

,Resolution_Id,Resolution
0,0,Open or Active
1,1,Cite or Arrest Adult
2,2,Exceptional Adult
3,3,Unfounded


### Neighborhood Dimension table

In [363]:
#Init of a dataframe for the dimension table
df_neighborhood = df[['Analysis_Neighborhood']].copy()
df_neighborhood.drop_duplicates(inplace=True)
df_neighborhood.drop(labels=[0], inplace=True)
df_neighborhood.reset_index(inplace=True)
df_neighborhood.drop(columns=['index'], inplace=True)
df_neighborhood['Neighborhood_Id'] = df_neighborhood.index
df_neighborhood = df_neighborhood[['Neighborhood_Id', 'Analysis_Neighborhood']]
df_neighborhood.rename(columns={'Analysis_Neighborhood': 'Neighborhood'}, inplace=True)

In [364]:
df_neighborhood.head()

,Neighborhood_Id,Neighborhood
0,0,Golden Gate Park
1,1,Financial District/South Beach
2,2,Potrero Hill
3,3,Tenderloin
4,4,Pacific Heights


### Location Dimension table

In [365]:
#Init of a dataframe for the dimension table
df_location = df[['Latitude','Longitude','Point']].copy()
df_location.drop_duplicates(inplace=True)
df_location.drop(labels=[0], inplace=True)
df_location.reset_index(inplace=True)
df_location.drop(columns=['index'], inplace=True)
df_location['Location_Id'] = df_location.index
df_location = df_location[['Location_Id', 'Latitude', 'Longitude', 'Point']]

In [366]:
df_location

,Location_Id,Latitude,Longitude,Point
0,0,37.772895,-122.454285,POINT (-122.45428511766733 37.772895177200766)
1,1,37.787359,-122.408227,POINT (-122.40822672700406 37.78735926098589)
2,2,37.762290,-122.401324,POINT (-122.40132418490647 37.76228996810526)
3,3,37.787038,-122.418271,POINT (-122.41827098126804 37.787037946181535)
4,4,37.793977,-122.429804,POINT (-122.42980398313114 37.79397724418211)
...,...,...,...,...
6471,6471,37.829264,-122.372825,POINT (-122.37282510300417 37.82926354769518)
6472,6472,37.755131,-122.468842,POINT (-122.46884229362875 37.75513114986906)
6473,6473,37.737977,-122.447541,POINT (-122.44754107619549 37.73797690612197)
6474,6474,37.713697,-122.411604,POINT (-122.4116044737096 37.71369702075132)


### Intersection Dimension table

In [367]:
#Init of a dataframe for the dimension table
df_intersection = df[['Intersection']].copy()
df_intersection.drop_duplicates(inplace=True)
df_intersection.drop(labels=[0], inplace=True)
df_intersection.reset_index(inplace=True)
df_intersection.drop(columns=['index'], inplace=True)
df_intersection['Intersection_Id'] = df_intersection.index
df_intersection = df_intersection[['Intersection_Id', 'Intersection']]

In [368]:
df_intersection.head()

,Intersection_Id,Intersection
0,0,STANYAN ST \ HAYES ST
1,1,GEARY ST \ POWELL ST
2,2,18TH ST \ DE HARO ST
3,3,POST ST \ LARKIN ST
4,4,LAGUNA ST \ PACIFIC AVE


### Fact Database

In [369]:
#Init of a dataframe for the Fact database table
df_fact_table = df.copy()

#Converting boolean column to binary column
#df_fact_table['Filed_Online'] = df_fact_table['Filed_Online'].notnull().astype(int)


df_fact_table['Incident_Time_Id'] = df_fact_table['Incident_Datetime'].dt.time.map(df_time.set_index('Time')['Id'])
df_fact_table['Incident_Date_Id'] = df_fact_table['Incident_Datetime'].dt.date.map(df_date.set_index('Date')['Id'])

df_fact_table['Report_Time_Id'] = df_fact_table['Report_Datetime'].dt.time.map(df_time.set_index('Time')['Id'])
df_fact_table['Report_Date_Id'] = df_fact_table['Report_Datetime'].dt.date.map(df_date.set_index('Date')['Id'])

df_fact_table['Intersection_Id'] = df_fact_table['Intersection'].map(df_intersection.set_index('Intersection')['Intersection_Id'])
df_fact_table['Police_district_Id'] = df_fact_table['Police_District'].map(df_police_district.set_index('Police_District')['Police_district_Id'])
df_fact_table['Resolution_Id'] = df_fact_table['Resolution'].map(df_resolution.set_index('Resolution')['Resolution_Id'])
df_fact_table['Neighborhood_Id'] = df_fact_table['Analysis_Neighborhood'].map(df_neighborhood.set_index('Neighborhood')['Neighborhood_Id'])

df_fact_table = df_fact_table.merge(df_location, on=['Latitude', 'Longitude', 'Point'], how='left')


#dropping the unwanted columns
df_fact_table.drop(columns=['CAD_Number', 'CNN', 'Invest_In_Neighborhoods_(IIN)_Areas', 'ESNCAG_-_Boundary_File', 'Central_Market/Tenderloin_Boundary_Polygon_-_Updated', 'Civic_Center_Harm_Reduction_Project_Boundary', 
                 'HSOC_Zones_as_of_2018-06-05', 'Supervisor_District', 'Supervisor_District_2012', 'Latitude', 'Latitude', 'Point', 'Row_ID', 'Incident_Datetime', 'Incident_Date', 'Incident_Time', 'Incident_Year',
                 'Incident_Day_of_Week', 'Report_Datetime', 'Report_Type_Description', 'Incident_Category', 'Incident_Subcategory', 'Incident_Description', 'Resolution','Intersection', 'Police_District', 'Analysis_Neighborhood',
                 'Longitude', 'Neighborhoods', 'Current_Supervisor_Districts', 'Current_Police_Districts'], inplace=True)


df_fact_table.dropna(inplace=True)

#adding columns for the primary key
df_fact_table.reset_index(inplace=True)
df_fact_table.drop(columns=['index'], inplace=True)
df_fact_table['Id'] = df_fact_table.index


#to order the columns in fact database
df_fact_table = df_fact_table[['Id', 'Incident_ID', 'Incident_Number', 'Incident_Date_Id', 'Incident_Time_Id', 'Report_Date_Id', 'Report_Time_Id', 'Incident_Code', 'Report_Type_Code', 'Filed_Online', 'Police_district_Id', 'Resolution_Id',
                               'Location_Id', 'Neighborhood_Id', 'Intersection_Id']]

#converting the column id's to integer type
df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']] = df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']].astype(int)


#dropping the unwanted columns
df_fact_table.drop(columns=['CAD_Number', 'CNN', 'Invest_In_Neighborhoods_(IIN)_Areas', 'ESNCAG_-_Boundary_File', 'Central_Market/Tenderloin_Boundary_Polygon_-_Updated', 'Civic_Center_Harm_Reduction_Project_Boundary', 
                 'HSOC_Zones_as_of_2018-06-05', 'Supervisor_District', 'Supervisor_District_2012', 'Latitude', 'Latitude', 'Point', 'Row_ID', 'Incident_Datetime', 'Incident_Date', 'Incident_Time', 'Incident_Year',
                 'Incident_Day_of_Week', 'Report_Datetime', 'Report_Type_Description', 'Incident_Category', 'Incident_Subcategory', 'Incident_Description', 'Resolution','Intersection', 'Police_District', 'Analysis_Neighborhood',
                 'Longitude', 'Neighborhoods', 'Current_Supervisor_Districts', 'Current_Police_Districts'], inplace=True)


#to order the columns in fact database
df_fact_table = df_fact_table[['Id', 'Incident_ID', 'Incident_Number', 'Incident_Date_Id', 'Incident_Time_Id', 'Report_Date_Id', 'Report_Time_Id', 'Filed_Online', 'Incident_Code', 'resolution_Id', 'Police_district_Id', 
                               'Location_Id', 'Neighborhood_Id', 'Location_Id']]

In [370]:
df_fact_table.head()

,Id,Incident_ID,Incident_Number,Incident_Date_Id,Incident_Time_Id,Report_Date_Id,Report_Time_Id,Incident_Code,Report_Type_Code,Filed_Online,Police_district_Id,Resolution_Id,Location_Id,Neighborhood_Id,Intersection_Id
0,0,1254318,230182844,1895,840,1899,681,4134,II,0,3,0,0,0,0
1,1,1254393,230184129,1638,720,1899,1040,71000,II,0,2,0,1,1,1
2,2,1254826,230187101,1900,1050,1900,1082,4134,II,0,4,0,2,2,2
3,3,1256563,230199764,1905,950,1905,961,51040,II,0,7,0,3,3,3
4,4,1062676,210537297,1329,580,1329,580,62071,II,0,7,0,4,4,4


In [371]:
df.head()

,Incident_Datetime,Incident_Date,Incident_Time,Incident_Year,Incident_Day_of_Week,Report_Datetime,Row_ID,Incident_ID,Incident_Number,CAD_Number,Report_Type_Code,Report_Type_Description,Filed_Online,Incident_Code,Incident_Category,Incident_Subcategory,Incident_Description,Resolution,Intersection,CNN,Police_District,Analysis_Neighborhood,Supervisor_District,Supervisor_District_2012,Latitude,Longitude,Point,Neighborhoods,ESNCAG_-_Boundary_File,Central_Market/Tenderloin_Boundary_Polygon_-_Updated,Civic_Center_Harm_Reduction_Project_Boundary,HSOC_Zones_as_of_2018-06-05,Invest_In_Neighborhoods_(IIN)_Areas,Current_Supervisor_Districts,Current_Police_Districts,Id
0,2023-03-13 23:41:00,2023/03/13,23:41,2023,Monday,2023-03-13 23:41:00,125373607041,1253736,230167874,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2023-03-01 05:02:00,2023/03/01,05:02,2023,Wednesday,2023-03-11 15:40:00,125379506374,1253795,236046151,NaN,CI,Coplogic Initial,1,6374,Property Crimes,Larceny Theft - Other,"Theft, Other Property, >$950",Open or Active,NaN,NaN,Mission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,2023-03-13 13:16:00,2023/03/13,13:16,2023,Monday,2023-03-13 13:17:00,125357107041,1253571,220343896,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,2023-03-13 10:59:00,2023/03/13,10:59,2023,Monday,2023-03-13 11:00:00,125355107041,1253551,230174885,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2023-03-14 18:44:00,2023/03/14,18:44,2023,Tuesday,2023-03-14 18:45:00,125402407041,1254024,230176728,NaN,VS,Vehicle Supplement,0,7041,Vehicle Theft and Recovery,Recovered Vehicle,"Vehicle, Recovered, Auto",Open or Active,NaN,NaN,Out of SF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [372]:
df_fact_table.columns

Index(['Id', 'Incident_ID', 'Incident_Number', 'Incident_Date_Id',
       'Incident_Time_Id', 'Report_Date_Id', 'Report_Time_Id', 'Incident_Code',
       'Report_Type_Code', 'Filed_Online', 'Police_district_Id',
       'Resolution_Id', 'Location_Id', 'Neighborhood_Id', 'Intersection_Id'],
      dtype='object')

In [373]:
df_fact_table.shape

(730522, 15)

df_fact_table.dropna(inplace=True)
df_fact_table.shape

df_fact_table['Time_Id'] = df_fact_table['Incident_Time'].map(df_time)

df_fact_table['Incident_Time_Id'] = df_fact_table['Incident_Datetime'].dt.time.map(df_time.set_index('time')['id'])
df_fact_table['Incident_Date_Id'] = df_fact_table['Incident_Datetime'].dt.date.map(df_date.set_index('date')['id'])

df_fact_table['Report_Time_Id'] = df_fact_table['Report_Datetime'].dt.time.map(df_time.set_index('time')['id'])
df_fact_table['Report_Date_Id'] = df_fact_table['Report_Datetime'].dt.date.map(df_date.set_index('date')['id'])

df_fact_table['Intersection_Id'] = df_fact_table['Intersection'].map(df_intersection.set_index('Intersection')['Intersection_Id'])

df_fact_table['Police_district_Id'] = df_fact_table['Police_District'].map(df_police_district.set_index('Police_District')['Police_district_Id'])

df_fact_table['resolution_Id'] = df_fact_table['Resolution'].map(df_resolution.set_index('Resolution')['resolution_Id'])

df_fact_table['Neighborhood_Id'] = df_fact_table['Analysis_Neighborhood'].map(df_neighborhood.set_index('Analysis_Neighborhood')['Neighborhood_Id'])

df_fact_table['Neighborhood_Id'] = df_fact_table['Analysis_Neighborhood'].map(df_neighborhood.set_index('Analysis_Neighborhood')['Neighborhood_Id'])

df_fact_table = df_fact_table.merge(df_location, on=['Latitude', 'Longitude', 'Point'], how='left')


#Converting boolean column to binary column
df_fact_table['Filed_Online'] = df_fact_table['Filed_Online'].notnull().astype(int)

In [375]:
df_fact_table.head()

,Id,Incident_ID,Incident_Number,Incident_Date_Id,Incident_Time_Id,Report_Date_Id,Report_Time_Id,Incident_Code,Report_Type_Code,Filed_Online,Police_district_Id,Resolution_Id,Location_Id,Neighborhood_Id,Intersection_Id
0,0,1254318,230182844,1895,840,1899,681,4134,II,0,3,0,0,0,0
1,1,1254393,230184129,1638,720,1899,1040,71000,II,0,2,0,1,1,1
2,2,1254826,230187101,1900,1050,1900,1082,4134,II,0,4,0,2,2,2
3,3,1256563,230199764,1905,950,1905,961,51040,II,0,7,0,3,3,3
4,4,1062676,210537297,1329,580,1329,580,62071,II,0,7,0,4,4,4


{"datetime_dim":datetime_dim.to_dict(orient="dict"),
    "passenger_count_dim":passenger_count_dim.to_dict(orient="dict"),
    "trip_distance_dim":trip_distance_dim.to_dict(orient="dict"),
    "rate_code_dim":rate_code_dim.to_dict(orient="dict"),
    "pickup_location_dim":pickup_location_dim.to_dict(orient="dict"),
    "dropoff_location_dim":dropoff_location_dim.to_dict(orient="dict"),
    "payment_type_dim":payment_type_dim.to_dict(orient="dict"),
    "fact_table":fact_table.to_dict(orient="dict")}

df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']] = df_fact_table[['Location_Id', 'Neighborhood_Id', 'Intersection_Id']].astype(int)

In [377]:
df_fact_table.dtypes

Id                     int64
Incident_ID            int64
Incident_Number        int64
Incident_Date_Id       int64
Incident_Time_Id       int64
Report_Date_Id         int64
Report_Time_Id         int64
Incident_Code          int64
Report_Type_Code      object
Filed_Online           int32
Police_district_Id     int64
Resolution_Id          int64
Location_Id            int32
Neighborhood_Id        int32
Intersection_Id        int32
dtype: object

In [378]:
{"date_dim":df_date.to_dict(orient="dict"),
    "time_dim":df_time.to_dict(orient="dict"),
    "report_info_dim":df_report_info.to_dict(orient="dict"),
    "location_dim":df_location.to_dict(orient="dict"),
    "intersection_dim":df_intersection.to_dict(orient="dict"),
    "neighborhood_dim":df_neighborhood.to_dict(orient="dict"),
    "resolution_dim":df_resolution.to_dict(orient="dict"),
    "police_district_dim":df_police_district.to_dict(orient="dict"),
    "incident_info_dim":df_incident_info.to_dict(orient="dict"),
    "crime_incidents_fact_table":df_fact_table.to_dict(orient="dict")}

{'date_dim': {'Id': {0: 0,
   1: 1,
   2: 2,
   3: 3,
   4: 4,
   5: 5,
   6: 6,
   7: 7,
   8: 8,
   9: 9,
   10: 10,
   11: 11,
   12: 12,
   13: 13,
   14: 14,
   15: 15,
   16: 16,
   17: 17,
   18: 18,
   19: 19,
   20: 20,
   21: 21,
   22: 22,
   23: 23,
   24: 24,
   25: 25,
   26: 26,
   27: 27,
   28: 28,
   29: 29,
   30: 30,
   31: 31,
   32: 32,
   33: 33,
   34: 34,
   35: 35,
   36: 36,
   37: 37,
   38: 38,
   39: 39,
   40: 40,
   41: 41,
   42: 42,
   43: 43,
   44: 44,
   45: 45,
   46: 46,
   47: 47,
   48: 48,
   49: 49,
   50: 50,
   51: 51,
   52: 52,
   53: 53,
   54: 54,
   55: 55,
   56: 56,
   57: 57,
   58: 58,
   59: 59,
   60: 60,
   61: 61,
   62: 62,
   63: 63,
   64: 64,
   65: 65,
   66: 66,
   67: 67,
   68: 68,
   69: 69,
   70: 70,
   71: 71,
   72: 72,
   73: 73,
   74: 74,
   75: 75,
   76: 76,
   77: 77,
   78: 78,
   79: 79,
   80: 80,
   81: 81,
   82: 82,
   83: 83,
   84: 84,
   85: 85,
   86: 86,
   87: 87,
   88: 88,
   89: 89,
   90: 90,
 

In [379]:
df_fact_table.head()

,Id,Incident_ID,Incident_Number,Incident_Date_Id,Incident_Time_Id,Report_Date_Id,Report_Time_Id,Incident_Code,Report_Type_Code,Filed_Online,Police_district_Id,Resolution_Id,Location_Id,Neighborhood_Id,Intersection_Id
0,0,1254318,230182844,1895,840,1899,681,4134,II,0,3,0,0,0,0
1,1,1254393,230184129,1638,720,1899,1040,71000,II,0,2,0,1,1,1
2,2,1254826,230187101,1900,1050,1900,1082,4134,II,0,4,0,2,2,2
3,3,1256563,230199764,1905,950,1905,961,51040,II,0,7,0,3,3,3
4,4,1062676,210537297,1329,580,1329,580,62071,II,0,7,0,4,4,4


df_fact_table.reset_index(inplace=True)
df_fact_table.drop(columns=['index'], inplace=True)
#adding columns for the primary key
df_fact_table['Id'] = df_fact_table.index

In [381]:
df_fact_table.head()

,Id,Incident_ID,Incident_Number,Incident_Date_Id,Incident_Time_Id,Report_Date_Id,Report_Time_Id,Incident_Code,Report_Type_Code,Filed_Online,Police_district_Id,Resolution_Id,Location_Id,Neighborhood_Id,Intersection_Id
0,0,1254318,230182844,1895,840,1899,681,4134,II,0,3,0,0,0,0
1,1,1254393,230184129,1638,720,1899,1040,71000,II,0,2,0,1,1,1
2,2,1254826,230187101,1900,1050,1900,1082,4134,II,0,4,0,2,2,2
3,3,1256563,230199764,1905,950,1905,961,51040,II,0,7,0,3,3,3
4,4,1062676,210537297,1329,580,1329,580,62071,II,0,7,0,4,4,4


In [382]:
df_fact_table.shape

(730522, 15)

In [ ]:
"""
select f.Id, f.Incident_Id, f.Incident_Number, 
       dti.Date as Incident_Date, ti.Time as Incidnet_Time, 
       dtr.Date as Report_Date, tr.Time as Report_time,
       inc.Incident_Code, inc.Incident_Category, inc.Incident_Subcategory, inc.Incident_Description,
       rec.Report_Type_Description,
       f.Filed_Online,
       pd.Police_District,
       res.Resolution,
       loc.Latitude,loc.Longitude,loc.Point,
       nbh.Analysis_Neighborhood,
       itr.Intersection
from crime-analysis-399301.crime_us_dataengineering.crime_incidents_fact_table as f
JOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dti
ON f.Incident_Date_Id = dti.Id
JOIN crime-analysis-399301.crime_us_dataengineering.time_dim as ti
On f.Incident_Time_Id = ti.Id
JOIN crime-analysis-399301.crime_us_dataengineering.date_dim as dtr
ON f.Report_Date_Id = dtr.Id
JOIN crime-analysis-399301.crime_us_dataengineering.time_dim as tr
On f.Report_Time_Id = tr.Id
JOIN crime-analysis-399301.crime_us_dataengineering.incident_info_dim as inc
ON f.Incident_Code = inc.Incident_Code
JOIN crime-analysis-399301.crime_us_dataengineering.report_info_dim as rec
ON f.Report_Type_Code = rec.Report_Type_Code
JOIN crime-analysis-399301.crime_us_dataengineering.police_district_dim as pd
ON f.Police_district_Id = pd.Police_district_Id
JOIN crime-analysis-399301.crime_us_dataengineering.resolution_dim as res
On f.Resolution_Id = res.Resolution_Id
JOIN crime-analysis-399301.crime_us_dataengineering.location_dim as loc
ON f.Location_Id = loc.Location_Id
JOIN crime-analysis-399301.crime_us_dataengineering.neighborhood_dim as nbh
ON f.Neighborhood_Id = nbh.Neighborhood_Id
JOIN crime-analysis-399301.crime_us_dataengineering.intersection_dim as itr
ON f.Intersection_Id = itr.Intersection_Id
"""


In [384]:
#Joining the fact and dimension table to check the due diligence of the data modelling
# Join the DataFrames without suffixes and handle duplicate columns manually
result_df = df_fact_table.merge(df_date[['Id', 'Date']], left_on='Incident_Date_Id', right_on='Id', how='inner') \
            .merge(df_time[['Id', 'Time']], left_on='Incident_Time_Id', right_on='Id', how='inner') \
            .merge(df_date[['Id', 'Date']], left_on='Report_Date_Id', right_on='Id', how='inner', suffixes=('_dtr', '_tr')) \
            .merge(df_time[['Id', 'Time']], left_on='Report_Time_Id', right_on='Id', how='inner', suffixes=('_dtr', '_tr')) \
            .merge(df_incident_info[['Incident_Code', 'Incident_Category', 'Incident_Subcategory', 'Incident_Description']], on='Incident_Code', how='inner') \
            .merge(df_report_info[['Report_Type_Code', 'Report_Type_Description']], left_on='Report_Type_Code', right_on='Report_Type_Code', how='inner') \
            .merge(df_police_district[['Police_district_Id', 'Police_District']], left_on='Police_district_Id', right_on='Police_district_Id', how='inner') \
            .merge(df_resolution[['Resolution_Id', 'Resolution']], left_on='Resolution_Id', right_on='Resolution_Id', how='inner') \
            .merge(df_location[['Location_Id', 'Latitude', 'Longitude', 'Point']], left_on='Location_Id', right_on='Location_Id', how='inner') \
            .merge(df_neighborhood[['Neighborhood_Id', 'Neighborhood']], left_on='Neighborhood_Id', right_on='Neighborhood_Id', how='inner') \
            .merge(df_intersection[['Intersection_Id', 'Intersection']], left_on='Intersection_Id', right_on='Intersection_Id', how='inner')


# Rename columns if needed to avoid duplicates
result_df.rename(columns={'Date': 'Incident_Date', 'Time': 'Incident_Time', 'Date_dtr': 'Report_Date', 'Time_dtr': 'Report_Time'}, inplace=True)

# Sort by 'Id' column
result_df.sort_values(by='Id', inplace=True)

# Reset the index
#result_df = result_df.reset_index(drop=True)



In [385]:
result_df.shape

(730522, 34)